In [1]:
from http.server import SimpleHTTPRequestHandler, HTTPServer, BaseHTTPRequestHandler
import sys
import requests
from urllib.parse import urljoin, unquote
import urllib.parse
import numpy as np
import keras 
from keras.models import  load_model

In [2]:
!pip install --upgrade keras

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.1 MB 465.5 kB/s eta 0:00:03
   ------ --------------------------------- 0.2/1.1 MB 1.0 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.1 MB 1.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.1 MB 1.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.1 MB 1.2 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.1 MB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.1 MB 1.2 MB/s eta 0:00:01
   ------------------ --------------------- 0.5/1.1 MB 1.2 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.1 MB 1.2 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.1 MB 1.3 MB/s eta 0:00:01
   --------------------------- 

In [3]:
# def load_data(urls, max_length=1000):
#     urls = [s.lower() for s in urls]
#     url_list = []
#     for url in urls:
#         # url decode
#         decoded_url = url_decode(url)
#         # unicode encode
#         encoded_url = [ord(x) for x in str(decoded_url).strip()]
#         encoded_url = encoded_url[:max_length]
#         url_len = len(encoded_url)
#         if url_len < max_length:
#             # zero padding
#             encoded_url += ([0] * (max_length - url_len))
#         url_list.append((encoded_url))
#     # convert to numpy array
#     url_list = np.array(url_list)
#     return url_list


def load_data(url, max_length=1000):
    # Chuyển đổi URL thành chữ thường
    url = url.lower()
    
    # Giải mã URL
    #decoded_url = url_decode(url)
    decoded_url = unquote(url)
    # Mã hóa Unicode
    encoded_url = [ord(x) for x in str(decoded_url).strip()]
    
    # Cắt ngắn URL nếu nó quá dài
    encoded_url = encoded_url[:max_length]
    
    # Thêm đệm 0 nếu cần
    url_len = len(encoded_url)
    if url_len < max_length:
        encoded_url += [0] * (max_length - url_len)
    
    # Chuyển đổi thành mảng numpy và trả về
    return np.array([encoded_url])

### Load Model 

In [4]:
path = r"E:\Nam3\BMW\project\NT213.O21.ANTT\Architecture B\TestCase[3,4,5,6]\model.h5"
model = load_model(path)

In [5]:
def predict(model ,url):
    y_pred = model.predict(url)
    print(y_pred)
    return 1 if y_pred >= 0.5 else 0

In [6]:
# class SimpleHttpProxy(BaseHTTPRequestHandler):
#     proxy_routes = {}

#     @classmethod
#     def set_routes(cls, proxy_routes):
#         cls.proxy_routes = proxy_routes

#     def do_GET(self):
#         try:
#             proxy_route = self.proxy_routes.get('proxy_route')
#             if proxy_route:
#                 target_url = urljoin(proxy_route, self.path)
#                 #print(load_data(target_url))
#                 y_pred = predict(model, load_data(target_url))
#                 print(y_pred)
#                 response = requests.get(target_url)
#                 self.send_response(response.status_code)
#                 self.send_header('Content-type', 'text/html')
#                 self.end_headers()
#                 self.wfile.write(response.content)
#             else:
#                 self.send_error(404, 'Proxy route not found')
#         except Exception as e:
#             self.send_error(500, str(e))

In [7]:
# def main():
#     # Set proxy route
#     SimpleHttpProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})
    
#     # Start HTTP server
#     server_address = ('127.0.0.1', 12345)
#     httpd = HTTPServer(server_address, SimpleHttpProxy)
#     print(f'Starting HTTP Proxy server on {server_address[0]}:{server_address[1]}')
#     try:
#         httpd.serve_forever()
#     except KeyboardInterrupt:
#         print('Keyboard interrupt, exiting.')

# if __name__ == '__main__':
#     main()

In [9]:
from http.server import BaseHTTPRequestHandler
from urllib.parse import urljoin
import requests
import re

class SimpleHttpProxy(BaseHTTPRequestHandler):
    proxy_routes = {}
    global_cookies = {}
    @classmethod
    def set_routes(cls, proxy_routes):
        cls.proxy_routes = proxy_routes

    def do_GET(self):
        try:
            proxy_route = self.proxy_routes.get('proxy_route')
            #print("proxy_route: ", proxy_route)
            if proxy_route:
                target_url = urljoin(proxy_route, self.path)
                #print(load_data(target_url))
                y_pred = predict(model, load_data(target_url))
                # print(f"Prediction cho {target_url}", y_pred)
                # proxies = {"http":"http://127.0.0.1:8081", "https":"https://127.0.0.1:8081"}
                #session = requests.Session()
                #print("cookie: ", requests.cookies)
                # print("self.global_cookies: ",self.global_cookies)
                if self.global_cookies:
                    # response = requests.get(target_url, proxies=proxies, allow_redirects=False, cookies=self.global_cookies)
                    response = requests.get(target_url, allow_redirects=False, cookies=self.global_cookies)
                else:
                    # response = requests.get(target_url, proxies=proxies, allow_redirects=False)  # Tắt chế độ tự động chuyển hướng
                    response = requests.get(target_url,  allow_redirects=False)  # Tắt chế độ tự động chuyển hướng
                #response = requests.get(target_url, allow_redirects=False)
                # print("test code22")
                if response.status_code in [301, 302, 303, 307, 308]:  # Kiểm tra nếu phản hồi là mã chuyển hướng
                    redirect_url = response.headers['Location']  # Lấy URL chuyển hướng từ header
                    response = requests.get(redirect_url)  # Gửi yêu cầu GET đến URL chuyển hướng
                self.send_response(response.status_code)
                # print("test code")
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(response.content)
            else:
                self.send_error(404, 'Proxy route not found')
        except Exception as e:
            self.send_error(500, str(e))


    def do_POST(self):
        try:
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            print("POST data:", post_data.decode('utf-8'))
            proxy_route = self.proxy_routes.get('proxy_route')
            # print("proxy_route", proxy_route)
            if proxy_route:
                target_url = urljoin(proxy_route, self.path)
                
                y_pred = predict(model, load_data(target_url))
                # print(f"Prediction cho {target_url}", y_pred)
                #print(y_pred)
                #print("target_url: ", target_url)
                #print("post_data:", post_data)
                headers= {"Content-Type": "application/x-www-form-urlencoded"}
                # proxies = {"http":"http://127.0.0.1:8081", "https":"https://127.0.0.1:8081"}
                session = requests.Session()
                # if "/dvwa/login.php" in target_url:
                #     response = session.post(target_url, data=post_data, headers=headers, proxies=proxies)
                #     redirect_url = response.headers.get('Location')
                #     if redirect_url:
                #         # Forward a GET request to the redirection URL
                #         response = session.get(redirect_url)
                #     global_session = response.cookies
                if "/dvwa/login.php" in target_url:
                    # response = session.post(target_url, data=post_data, headers=headers, proxies=proxies, allow_redirects=False)
                    response = session.post(target_url, data=post_data, headers=headers, allow_redirects=False)
                    if response.status_code == 302:
                        cookies_dict = requests.utils.dict_from_cookiejar(response.cookies)
                        #print("cookies_dict:" ,cookies_dict)
                    #print("response: ",response.status_code)
                    # Check if the response contains a redirection URL
                    redirect_url = response.headers.get('Location')
                    if redirect_url:
                        # Forward a GET request to the redirection URL
                        response = session.get("http://192.168.112.135/dvwa/"+redirect_url)
                    self.global_cookies.update(cookies_dict)
                elif "/dvwa/security.php" in target_url:
                        # response = session.post(target_url, data=post_data, headers=headers,cookies=self.global_cookies, proxies=proxies, allow_redirects=False)
                        response = session.post(target_url, data=post_data, headers=headers,cookies=self.global_cookies, allow_redirects=False)
                        
                            #print("cookies_dict:" ,cookies_dict)
                        #print("response: ",response.status_code)
                        # Check if the response contains a redirection URL
                        self.global_cookies['security'] = response.cookies.get_dict().get('security')
                        self.global_cookies.update(self.global_cookies)
                        print("cookie: ", self.global_cookies)
                        redirect_url = response.headers.get('Location')
                        if redirect_url:
                            # Forward a GET request to the redirection URL
                            # response = session.get("http://192.168.112.135/"+redirect_url, proxies=proxies, cookies=self.global_cookies)
                            response = session.get("http://192.168.112.135/"+redirect_url,  cookies=self.global_cookies)
                        
                else:
                    if self.global_cookies:
                        # response = session.post(target_url, data=post_data, headers=headers,cookies=self.global_cookies, proxies=proxies, allow_redirects=False)
                        response = session.post(target_url, data=post_data, headers=headers,cookies=self.global_cookies, allow_redirects=False)
                        if response.status_code == 302:
                            cookies_dict = requests.utils.dict_from_cookiejar(response.cookies)
                            #print("cookies_dict:" ,cookies_dict)
                        #print("response: ",response.status_code)
                        # Check if the response contains a redirection URL
                        redirect_url = response.headers.get('Location')
                        if redirect_url:
                            # Forward a GET request to the redirection URL
                            response = session.get("http://192.168.112.135/"+redirect_url)
                #print("self.global_cookies:", self.global_cookies)
                #print("Response:", response.text)
                
                # Print session cookies
                #print("Session cookies:", response.cookies.get_dict())
                
                # Forward the response to the client without any additional processing
                self.send_response(response.status_code)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(response.content)
            else:
                self.send_error(404, 'Proxy route not found')
        except Exception as e:
            self.send_error(500, str(e))


def start_proxy_server(port=8080):
    from http.server import HTTPServer
    server_address = ('127.0.0.1', port)
    httpd = HTTPServer(server_address, SimpleHttpProxy)
    print(f'Starting HTTP Proxy server on 127.0.0.1:{port}')
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        print('Keyboard interrupt, exiting.')


# Set up proxy routes
SimpleHttpProxy.set_routes({'proxy_route': 'http://192.168.1.194/dvwa/'})
SimpleHttpProxy.set_routes({'proxy_route': 'http://demo.testfire.net/'})

# Start the proxy server
start_proxy_server()


Starting HTTP Proxy server on 127.0.0.1:8080
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[[0.18570507]]


127.0.0.1 - - [10/Apr/2024 15:56:26] "GET http://192.168.112.135/dvwa/login.php HTTP/1.1" 200 -


POST data: username=admin&password=password&Login=Login
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
[[0.18570507]]


127.0.0.1 - - [10/Apr/2024 15:56:33] "POST http://192.168.112.135/dvwa/login.php HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[[0.9448092]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


127.0.0.1 - - [10/Apr/2024 16:33:47] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [10/Apr/2024 16:33:47] "CONNECT aus5.mozilla.org:443 HTTP/1.1" 501 -
127.0.0.1 - - [10/Apr/2024 16:33:47] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [10/Apr/2024 16:33:47] "CONNECT incoming.telemetry.mozilla.org:443 HTTP/1.1" 501 -


[[0.00209939]]


127.0.0.1 - - [10/Apr/2024 16:33:48] "GET http://ciscobinary.openh264.org/openh264-win64-2e1774ab6dc6c43debb0b5b628bdf122a391d521.zip HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2024 16:33:48] code 500, message 'utf-8' codec can't decode byte 0xe4 in position 23: invalid continuation byte
127.0.0.1 - - [10/Apr/2024 16:33:48] "POST http://ocsp.digicert.com/ HTTP/1.1" 500 -
127.0.0.1 - - [10/Apr/2024 16:34:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [10/Apr/2024 16:34:04] "CONNECT d1gm2ml4nzotep.cloudfront.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [10/Apr/2024 16:34:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [10/Apr/2024 16:34:04] "CONNECT d1gm2ml4nzotep.cloudfront.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [10/Apr/2024 16:34:04] code 501, message Unsupported method ('CONNECT')
127.0.0.1 - - [10/Apr/2024 16:34:04] "CONNECT a703.l461.r761.fastcloudcdn.net:443 HTTP/1.1" 501 -
127.0.0.1 - - [10/Apr/2024 16:34:04] code 501, message Unsupported method ('CONNECT

Keyboard interrupt, exiting.


In [ ]:
# # Sử dụng hàm load_data với một URL
# url = "http://demo.testfire.net/search.jsp?query=%3Cscript%3Ealert%281%29%3C%2Fscript%3E"

# loaded_data